In [9]:
example = SensorValues(PIP=20, PEEP=5, FIO2 = 60,                           \
                       TEMP=37, HUMIDITY=60, VTE=3, BREATHS_PER_MINUTE=20,  \
                       INSPIRATION_TIME_SEC=1.5, TIMESTAMP=time.time()      ) 

NameError: name 'SensorValues' is not defined

Then this is supposed to work

In [7]:
import threading
import sys

sys.path.append('../vent/')
from ControlModuleBase import get_control_module

In [8]:
instance = get_control_module(sim_mode=True)

TypeError: __init__() missing 3 required positional arguments: 'fname', 'lname', and 'year'

In [ ]:
exampleControlSettings = ControlSettings(PIP=20, PEEP=5, BREATHS_PER_MINUTE=20, INSPIRATION_TIME=1, TIMESTAMP = time.time())

instance.set_controls(controlSettings)


In [ ]:



#Starts a thread running the controller job
controller = threading.Thread(target = instance.start(), args=(1,))
controller.start()


for t in np.arange(0,10,0.01):
    sleep(0.01)
    # start running
    # controls actuators to achieve target state
    # determined by settings and assessed by sensor values

    instance.get_sensor_values()
    # returns SensorValues, include a timestamp and loop counter

    instance.get_alarms()
    instance.get_active_alarms()
    instance.get_logged_alarms()
    
    print(instance.recvUIHeartbeat())  #test that is alive

instance.stop()

